<table>
 <tr align=left><td><img align=left src="https://i.creativecommons.org/l/by/4.0/88x31.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [ ]:
%matplotlib inline
import clawpack.clawutil.nbtools as nbtools
import clawpack.visclaw.JSAnimation.JSAnimation_frametools as J
from IPython.display import Image

# Storm Surge Tutorial - Katrina

This notebook is a tutorial that will walk you through setting up a storm surge simulation in GeoClaw.  Instead of covering all the possible setup and modifications we will focus on storm surge specific topics and refer to other tutorials for discussion of topics that are generic to any GeoClaw simulation to the other tutorials based on the simulation of the Chile 2010 tsunami.  Note that this notebook is mostly intended to be a guide while working at the command-line.  

## Setup

Setting up the simulation requires a number of inputs, most of which are common between all GeoClaw simulations.  Here we are going to focus on parameters that are either specific to storm surge or bear special note when simulating storm surge.  As we go through these settings please follow along in the `setrun.py` file.

### Landfall Time

First off though we should cover the use of the variable `katrina_landfall` throughout the `setrun.py` file.  At the very top of the file we see this parameter set to
```python
katrina_landfall = datetime.datetime(2005, 8, 29, 6) - datetime.datetime(2005, 1, 1, 0).
```
This is used to approximately specify the landfall (1110 UTC, 6:10 a.m. CDT on Monday, August 29, 2005) of the hurricane relative to the beginning of the year which is needed so that the storm data can be easily related to this value.  It then is used to specify the timing for the beginning of the simulation in
```python
clawdata.t0 = days2seconds(katrina_landfall.days - 2) + katrina_landfall.seconds
```
the final time of the simulation via
```python
clawdata.tfinal = days2seconds(katrina_landfall.days + 2) + katrina_landfall.seconds
```
and the number of output files via
```python
recurrence = 24
clawdata.num_output_times = int((clawdata.tfinal - clawdata.t0) * recurrence / (60**2 * 24))
```

### The Storm

Now that we have specified the landfall we can move further down the `setrun` file where we see the function `set_storm` which sets various storm related parameters in the object `surge_data`.

First off we see two physics based parameters
```python
data.rho_air = 1.15
data.ambient_pressure = 101.3e3 
```
which more than likely do not need to be changed when moving between storm surge simulations.  These parameters are used when computing the wind and pressure fields and their impact on the water.

The next three parameters control and turn on and off the wind and pressure forcing
```python
data.wind_forcing = True
data.pressure_forcing = True
```

The wind forcing in GeoClaw currently takes the form
$$
    \rho_{\text{air}} c_f |W| W
$$
where $W$ is the wind velocity.  The determination of the friction coefficient $c_f$ is determined by the parameter
```python
data.drag_law = 2
```
for which valid values can be 
 - `0` for no drag,
 - `1` for Garret wind drag, and
 - `2` for Powell wind drag.

The Garret wind drag takes the form
$$
    c_f = \min(W_\text{limit}, 0.75 + 0.067 + W) \times 10^{-3}
$$
where as Powell uses a similar approach but is sector based as detailed in

M.D. Powell (2006). “Final Report to the National Oceanic and 
  Atmospheric Administration (NOAA) Joint Hurricane Testbed (JHT) 
  Program.” 26 pp.

The next parameters control refinement criteria related to the storm's location and computed wind speeds.  In this case we have set them to
```python
data.wind_refine = [20.0, 40.0, 60.0]
data.R_refine = [60.0e3, 40e3, 20e3]
```
The first parameter controls refinement based on wind speed in m/s while the second is based on distance to the eye of the storm.  Each of these are specified as a Python list such that refinement is forced to the next level if it surpasses the given parameter.  For instance if you are within 60 kilometers of the storm's eye refinement is forced to level, 40 kilometers is forced to level 3, and within 20 kilometers to level 4.

Finally we come to the specification of the storm itself with 
```python
data.storm_type = 1
data.landfall = days2seconds(katrina_landfall.days)         \
                + katrina_landfall.seconds
data.display_landfall_time = True
```
The first parameter sets the type of storm being specified of which `1` is the only relevant option today specifying a parameterized Holland reconstructed storm (in the future additional parameterized models as well as direct input from WRF models will be supported).  The second parameter is used to compute time offsets in seconds since the beginning of the year as covered above.  The third parameter tells GeoClaw to output the current time to the terminal in days relative to landfall rather than the default which is seconds from the beginning of the year.

The final parameter
```python
data.storm_file = os.path.expandvars(os.path.join(os.getcwd(), 'katrina.storm'))
```
is the path to the file that specifies the storm location and characteristics at a given time.  Currently supported formats include HURDAT1, HURDAT2 and JMA data.  In this case the format is in the old HURDAT1 format and looks like
```
AL, 12, 2005082812, 01, CARQ,   0, 257N,  877W, 140,  908, HU,  34, NEQ,  160,  160,  125,  140, 1008,  300,  20,   0,   0,   L,   0,   X, 300,  10,    KATRINA, D, 
```

### Friction

Variable friction can be important to accurately simulated overland flooding such as in a storm surge event.  In GeoClaw we compute friction impacts via a Manning's N formulation which takes the form
$$
hu \frac{g n^2}{h^{4/3}} \sqrt{u^2 + v^2}
$$
where the parameter $n$ represents a form of roughness of the bottom surface.  GeoClaw currently allows partial support for variable spatial specification of the parameter $n$ by specifying regions and iso-baths.  Take a look again at the `setrun.py` file where right after the function `set_storm` we have the function `set_friction`.

The first parameter turns on variable friction via
```python
data.variable_friction = True
```
Next are two regions that are specified.  The format of the list takes on the form of the region that the rule specified covers, the iso-baths between which the friction will be set, and finally the actual coefficients themselves.

The first is
```python
data.friction_regions.append([rundata.clawdata.lower,
                              rundata.clawdata.upper,
                              [np.infty, 0.0, -np.infty],
                              [0.030, 0.022]])
```
and sets a variable friction region that covers the entire domain and sets the friction to `0.030` below bathymetry/topography at `0.0` and to 0.022 to regions above `0.0`.  The second
```python
data.friction_regions.append([(-98, 25.25), (-90, 30),
                              [np.infty, -10.0, -200.0, -np.infty],
                              [0.030, 0.012, 0.022]])
```
only covers the region between longitudes (-98, -90) and latitudes (25.25, 30).  Here the friction is set to `0.03` above -10 meters, to `0.012` between -10 and -200 meters, and to `0.022` below -200 meters.  This latter rule was specified to represent the abnormally smooth Louisiana/Texas shelf region.



### Other Parameters of Note

One difference between tsunami and storm surge simulations is the number of auxiliary fields that need to be specified.  In most simulations this is set to 
```python
clawdata.num_aux = 7
```
representing the common GeoClaw fields related to bathymetry and geometry but adding on support for variable friction and the storage of the wind velocity and pressure.

For storm surge it is also often helpful to specify refinement based on water currents.  If you would like to do this as well the parameter of interest is
```python
refine_data.speed_tolerance = [1.0,2.0,3.0,4.0]
```
which behaves slightly differently than the `wave_tolerance` parameter.  Here refinement is triggered if the speed is above the given tolerance.  In this case level 2 refinement is triggered if the water current is above 1 m/s, level 3 for 2 m/s, etc.

## Analysis and Plotting

### Plotting

There are a couple of storm surge specific plotting options that are important to note.  Opening the `setplot.py` file we can observe some of these.  If you want to generate the plots run `make .plots` from the command line after running a simulation.

#### Landfall

As was the case in specifying the simulation the landfall time also makes an appearance here and is used to label time on any plots.  These are specified via
```python
landfall_dt = datetime.datetime(2005,8,29,6) - datetime.datetime(2005,1,1,0)
landfall = (landfall_dt.days - 1.0) * 24.0 * 60**2 + landfall_dt.seconds
```

#### Surge Plotting Module

A number of helpful functions that help to plot storm surge simulations is provided in the module `clawpack.geoclaw.surge.plot` found at the path `$CLAW/src/python/geoclaw/surge/plot.py`.  We will cover a couple of relevant functions in this module but leave many of them up to the interested reader.

First off at the top of the `setplot` file you will note the definition of the function `surge_afteraxes` via
```python
surge_afteraxes = lambda cd: surge.surge_afteraxes(cd, track, landfall, plot_direction=False)
```
which is used for most of the plots produced.  This function plots the location of the eye of the storm, the direction of the storm, and modifies the title to days relative to landfall.  The reason the function is redefined in the `setplot` file is that we want to hand it the track of the storm which is read in at the top of the `setplot` file.

Additionally the surge plotting module contains a number of helper functions that create `plotitem`s including surface elevation, land, water speed, and friction.  Looking at the first plot description for the surface you will notice the three lines
```python
surge.add_surface_elevation(plotaxes, bounds=surface_limits, shrink=full_shrink)
surge.add_land(plotaxes, topo_min=-10.0, topo_max=5.0)
surge.add_bathy_contours(plotaxes)
```
which add plot items to plot the surface elevation, land, and a bathymetry contour respectively.

Along with the solution plotting you can also plot the wind speed and pressure via the functions
```python
surge.add_wind(plotaxes, bounds=wind_limits, plot_type='imshow')
surge.add_pressure(plotaxes, bounds=pressure_limits)
```
respectively.

## Further Exploration

### Output times

Change the number of output times to every 3 hours or every hour.

### Gauge specification

Pick out and place a new gauge that corresponds to an actual [observation point](https://tidesandcurrents.noaa.gov/map/).  We need to detide this data (do this if you can) but try to just add the point to the `setrun.py` file and plot the results.

### Improved Bathymetry 

We downloaded better bathymetry for the region around New Orleans but are not using it.  Try uncommenting the relevant lines in the `setrun.py` file and see what happens.  What happens when you change the refinement?  What about adding a refinement region or constraining the topography levels?

### Friction regions

Try modifying the friction regions.  Make sure to take a look and adjust the plotting of the friction regions in `setplot.py` to help out with this.

### Improved Plotting Regions

Try adding a plot that focuses on the New Orleans region maybe.  If you want a challenge try to plot where Tulane is on the resulting plot.